<a href="https://colab.research.google.com/github/osama1hatem2abdelkreem3/Graduation-Project/blob/Data-Augmantation/GP_without_Data_Augmantation1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install the dependeses

In [1]:
!pip install torchinfo

# import packages

In [28]:
import os
import sys
from collections import Counter

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import PIL
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from torch.utils.data import DataLoader, random_split
from torchinfo import summary
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

torch.backends.cudnn.deterministic = True

In [3]:
print("Platform:", sys.platform)
print("Python version:", sys.version)
print("---")
print("matplotlib version:", matplotlib.__version__)
print("pandas version:", pd.__version__)
print("PIL version:", PIL.__version__)
print("torch version:", torch.__version__)
print("torchvision version:", torchvision.__version__)

Platform: linux
Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
---
matplotlib version: 3.10.0
pandas version: 2.2.2
PIL version: 11.3.0
torch version: 2.8.0+cu126
torchvision version: 0.23.0+cu126


In [4]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f"Using {device} device.")

Using cuda device.


# Task
Download and extract the dataset from the Google Drive link "https://drive.google.com/file/d/1-vdiNStgmTHq4kF6zGzN2fYLI9qIz-tE/view".

## Install gdown

### Subtask:
Install the `gdown` library using pip.


**Reasoning**:
Install the gdown library using pip.



In [5]:
!pip install gdown

## Download the dataset

### Subtask:
Use `gdown` to download the zip file from the provided Google Drive link.


**Reasoning**:
Import gdown and download the file using gdown.download().



In [6]:
import gdown

file_id = "1-vdiNStgmTHq4kF6zGzN2fYLI9qIz-tE"
output_path = "plant_village.zip"

gdown.download(id=file_id, output=output_path, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1-vdiNStgmTHq4kF6zGzN2fYLI9qIz-tE
From (redirected): https://drive.google.com/uc?id=1-vdiNStgmTHq4kF6zGzN2fYLI9qIz-tE&confirm=t&uuid=e478b3d6-f8bf-48c9-a75b-7daf369d450e
To: /content/plant_village.zip
100%|██████████| 320M/320M [00:05<00:00, 55.9MB/s]


'plant_village.zip'

## Extract the dataset

### Subtask:
Extract the contents of the downloaded zip file.


**Reasoning**:
Extract the contents of the downloaded zip file.



In [7]:
import zipfile
import os

zip_file_path = "plant_village.zip"
extraction_directory = "plant_village"

os.makedirs(extraction_directory, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_directory)

print(f"Extracted {zip_file_path} to {extraction_directory}")

Extracted plant_village.zip to plant_village


# split folder into train and test

In [8]:
!pip install split-folder

In [9]:
import split_folder
input_folder = '/content/plant_village/SPAD_DATA'
output_folder = '/content/DataSet'
split_folder.ratio(input_folder, output=output_folder, ratio=(.8, .2))


Copying (82) of .. train/Electrical-damage



  0%|          | 0/82 [00:00<?, ?it/s]


Copying (21) of .. val/Electrical-damage



  0%|          | 0/21 [00:00<?, ?it/s]


Copying (154) of .. train/Clean



...

  0%|          | 0/38 [00:00<?, ?it/s]


Copying (55) of .. train/Physical-Damage



  0%|          | 0/55 [00:00<?, ?it/s]


Copying (14) of .. val/Physical-Damage



  0%|          | 0/14 [00:00<?, ?it/s]

In [10]:
%rm -rf /content/plant_village

# make the training tensor and the data augmantation

In [21]:
train_transform = transforms.Compose(
    [

        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomRotation(degrees=30),
        # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.RandomGrayscale(p=0.1),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
        transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
        transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
        transforms.ToTensor(),
    ]
)

## def the train and test files

In [22]:
data_dir = '/content/DataSet'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'val')

print("Will read data from", train_dir)

Will read data from /content/DataSet/train


In [30]:
dataset =datasets.ImageFolder(root = train_dir, transform = train_transform)

# classes

In [31]:
print("Classes:")
print(dataset.classes)
print(f"That's {len(dataset.classes)} classes")
print()
print("Tensor shape for one image:")
print(dataset[0][0].shape)

Classes:
['Bird-drop', 'Clean', 'Dusty', 'Electrical-damage', 'Physical-Damage', 'Snow-Covered']
That's 6 classes

Tensor shape for one image:
torch.Size([3, 224, 224])


In [32]:
batch_size = 32
dataset_loader = DataLoader(dataset, batch_size=batch_size)

# Get one batch
first_batch = next(iter(dataset_loader))

print(f"Shape of one batch: {first_batch[0].shape}")
print(f"Shape of labels: {first_batch[1].shape}")

Shape of one batch: torch.Size([32, 3, 224, 224])
Shape of labels: torch.Size([32])


In [33]:
def get_mean_std(loader):
    """Computes the mean and standard deviation of image data.

    Input: a `DataLoader` producing tensors of shape [batch_size, channels, pixels_x, pixels_y]
    Output: the mean of each channel as a tensor, the standard deviation of each channel as a tensor
            formatted as a tuple (means[channels], std[channels])"""

    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for data, _ in tqdm(loader, desc="Computing mean and std", leave=False):
        channels_sum += torch.mean(data, dim=[0, 2, 3])
        channels_squared_sum += torch.mean(data**2, dim=[0, 2, 3])
        num_batches += 1
    mean = channels_sum / num_batches
    std = (channels_squared_sum / num_batches - mean**2) ** 0.5

    return mean, std